In [1]:
%run imports.py
%matplotlib inline

In [2]:
from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard

In [3]:
import torchvision.models as models
mobilenet = models.mobilenet_v2(pretrained=True)
mobilenet

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [ ]:
# Windows
!rd /s "exps/exp1/logs"
# Linux
# !rm -r exps/exp1/logs

In [11]:
%tensorboard --logdir exps/exp1/logs

Reusing TensorBoard on port 6006 (pid 2332), started 0:01:26 ago. (Use '!kill 2332' to kill it.)

In [5]:
input_batch = torch.ones((64, 3, 32, 32))
writer = SummaryWriter(log_dir="exps/exp1/logs/mobilenet_graph")
writer.add_graph(mobilenet, input_batch)
writer.close()

In [6]:
# average input channels of conv layer and return new model
def replace_conv(model, inplanes, outplanes=32):
    conv_weights = list(model.parameters())[0].clone().detach().numpy()
    new_conv_weights = conv_weights.mean(axis=1)
    new_conv_weights = np.repeat(new_conv_weights[:, np.newaxis, :, :], inplanes, axis=1)
    new_conv = torch.nn.Conv2d(inplanes, outplanes, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    new_conv.weight = torch.nn.Parameter(torch.from_numpy(new_conv_weights))
    model._modules['features'][0][0] = new_conv
    return model

In [7]:
mobilenet = models.mobilenet_v2(pretrained=True)
mobilenet = replace_conv(mobilenet, 10)
input_batch = torch.ones((64, 10, 32, 32))
writer = SummaryWriter(log_dir="exps/exp1/logs/new_mobilenet_graph")
writer.add_graph(mobilenet, input_batch)
writer.close()

In [8]:
%%writefile exps/exp1/transforms.py

import torch
from torchvision import transforms, utils
import albumentations as alb
import albumentations.augmentations.transforms as aat

class AlbuWrapperNumpy:  # typing: ignore
    def __init__(self, atrans: alb.BasicTransform):
        self.atrans = atrans

    def __call__(self, img):
        return self.atrans(image=img)["image"]

alb_transforms = alb.Compose(
        [
            alb.Resize(256, 256, always_apply=True),
            alb.RandomCrop(244, 244, always_apply=True),
                aat.HorizontalFlip(),
            aat.Cutout(2, 10, 10)
        ])
alb_rescale = alb.Resize(244, 244, always_apply=True)

transform = transforms.Compose(
  [transforms.ToTensor(),
    transforms.Normalize(0.449, 0.226)])

test_transforms = transforms.Compose(
  [AlbuWrapperNumpy(alb_rescale), transform])
train_transforms = transforms.Compose(
  [AlbuWrapperNumpy(alb_transforms), transform])

Overwriting exps/exp1/transforms.py


In [9]:
%run exps/exp1/transforms.py

In [13]:
%run preprocessing/FallDetectionDataset.py

train_dataset = FallDetectionDataset(train=True, optical_flow=True, step_between_clips=10, transform=train_transforms)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, pin_memory = True,
                                          shuffle=True, num_workers=0)

test_dataset = FallDetectionDataset(train=False, optical_flow=True, step_between_clips=10, transform=train_transforms)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32,  pin_memory = True,
                                         shuffle=False, num_workers=0)
loss_function = torch.nn.modules.loss.CrossEntropyLoss()

[Dataset] reading folder:  data/FD/1219
[Dataset] Creating optical flow for folder:  data/FD/1219



[Dataset] reading folder:  data/FD/1260
[Dataset] Creating optical flow for folder:  data/FD/1260



[Dataset] reading folder:  data/FD/1301
[Dataset] Creating optical flow for folder:  data/FD/1301



[Dataset] reading folder:  data/FD/1378
[Dataset] Creating optical flow for folder:  data/FD/1378



[Dataset] reading folder:  data/FD/1392
[Dataset] Creating optical flow for folder:  data/FD/1392


...


[Dataset] reading folder:  data/FD/758
[Dataset] Creating optical flow for folder:  data/FD/758



[Dataset] reading folder:  data/FD/786
[Dataset] Creating optical flow for folder:  data/FD/786



[Dataset] reading folder:  data/FD/807
[Dataset] Creating optical flow for folder:  data/FD/807



[Dataset] reading folder:  data/FD/832
[Dataset] Creating optical flow for folder:  data/FD/832



[Dataset] reading folder:  data/FD/925
[Dataset] Creating optical flow for folder:  data/FD/925


In [30]:
def train(model, train_loader, optimizer, loss_function, epoch, writer, device):
    model = model.to(device)
    train_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        if batch_idx % 5 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    train_loss /= len(train_loader.dataset)
    writer.add_scalar("loss/train", train_loss, global_step=epoch)

In [31]:
def test(model, test_loader, loss_function, epoch, writer, device):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device)
            output = model(data)
            test_loss += loss_function(output, target).sum().item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    writer.add_scalar("loss/test", test_loss, global_step=epoch)
    writer.add_scalar("accuracy", accuracy, global_step=epoch)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        accuracy))
    return accuracy

In [32]:
def train_model(model, lr=0.1, epochs=5, start_epoch=0, device_name="cpu"):
    device = torch.device(device_name)
    model = model.to(device)
    writer = SummaryWriter(log_dir=f"exps/exp1/logs/device{device_name}/lr{lr}/")
    optim = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.00001)
    scheduler = torch.optim.lr_scheduler.StepLR(optim, 1, gamma=0.25)
    accuracy = 0
    tol = 0.005
    no_advance = 0
    print("Started training")
    for epoch in range(start_epoch, start_epoch + epochs):
        train(model, train_loader, optim, loss_function, epoch, writer, device)
        new_accuracy = test(model, test_loader, loss_function, epoch, writer, device)
        no_advance = no_advance + 1 if new_accuracy - accuracy <= tol else 0
        if no_advance >= 5:
            break
        accuracy = new_accuracy
        scheduler.step()
        print("Current LR: ", scheduler.get_lr())
    writer.close()
    return model

In [17]:
def get_model(freeze=True, new_classifier=True, inplanes=10, pretrained=True, start_model=None):
    model = start_model
    if start_model is None:
        model = models.mobilenet_v2(pretrained=pretrained)
    start_freeze = 0
    if new_classifier:
        model = replace_conv(model, inplanes)
        start_freeze = 1
    for layer in model.features[start_freeze:]:
        for p in layer.parameters():
            p.requires_grad = False
    input_batch = torch.ones((64, inplanes, 32, 32))
    writer = SummaryWriter(log_dir="exps/exp1/logs/model")
    writer.add_graph(model, input_batch)
    writer.close()
    return model

In [34]:
PATH = "exps/exp1/model.pth"
model = get_model(inplanes=20)
if os.path.isfile(PATH):
    model.load_state_dict(torch.load(PATH))
%time train_model(model, epochs=20, device_name="cuda")
torch.save(model.state_dict(), PATH)

Started training
Train Epoch: 0 [0/1800 (0%)]	Loss: 13.327008
Train Epoch: 0 [160/1800 (9%)]	Loss: 35.803833
Train Epoch: 0 [320/1800 (18%)]	Loss: 16.783487
Train Epoch: 0 [480/1800 (26%)]	Loss: 38.873627
Train Epoch: 0 [640/1800 (35%)]	Loss: 24.753510
Train Epoch: 0 [800/1800 (44%)]	Loss: 42.182766
Train Epoch: 0 [960/1800 (53%)]	Loss: 16.004786
Train Epoch: 0 [1120/1800 (61%)]	Loss: 6.237300
Train Epoch: 0 [1280/1800 (70%)]	Loss: 5.366728
Train Epoch: 0 [1440/1800 (79%)]	Loss: 7.472356
Train Epoch: 0 [1600/1800 (88%)]	Loss: 8.473408
Train Epoch: 0 [1760/1800 (96%)]	Loss: 4.223253

Test set: Average loss: 0.1520, Accuracy: 226/333 (68%)

Current LR:  [0.00625]
Train Epoch: 1 [0/1800 (0%)]	Loss: 4.421755
Train Epoch: 1 [160/1800 (9%)]	Loss: 3.202326
Train Epoch: 1 [320/1800 (18%)]	Loss: 2.657738
Train Epoch: 1 [480/1800 (26%)]	Loss: 7.726010
Train Epoch: 1 [640/1800 (35%)]	Loss: 2.231459
Train Epoch: 1 [800/1800 (44%)]	Loss: 3.444578
Train Epoch: 1 [960/1800 (53%)]	Loss: 1.239429
Train

In [38]:
writer = SummaryWriter(log_dir="exps/exp1/logs/model")
new_accuracy = test(model, test_loader, loss_function, 20, writer, device="cuda")
writer.close()


Test set: Average loss: 0.0663, Accuracy: 214/333 (64%)




Test set: Average loss: 0.0641, Accuracy: 215/333 (65%)

